In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk


In [2]:
classifier_id = 37

In [3]:
stack = 'MD653'

In [6]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
                argument_type='list2',
               cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 332.130402 seconds


In [7]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

# downscale = 8
downscale = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': downscale},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
                argument_type='list2',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

Resampling scoremaps ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 65.687401 seconds


In [10]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
# viz_downscale = 8
viz_downscale = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v4.py'),
                'stack': stack,
                 'classifier_id': classifier_id,
                'downscale': viz_downscale,
                'add_label_text': '-a' if add_label_text else ''},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
                argument_type='list2',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # For one stack, 75s

visualize scoremaps ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 60.658221 seconds


In [8]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

use_nissl_only = False
volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d %(use_nissl_only)s' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale,
                'use_nissl_only': '-n' if use_nissl_only else ''},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 116s

constructing score volumes ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 221.657870 seconds


In [11]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)

compute score volume gradients...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 30.809991 seconds


# Global alignment

In [75]:
classifier_setting = 37
warp_setting = 1

In [76]:
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 'VLL_L', 'VLL_R', '7n_L']
structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 'VLL_L', 'VLL_R', '7n_L',
          '7n_R', 'Tz_L', 'Tz_R', 'VCA_L', 'VCP_R']

In [15]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_num 5 --structures \'%(structures)s\'' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v3.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting,
                'structures': json.dumps(structure_subset)},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD652']),
                argument_type='single',
               cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 739.465696 seconds


In [77]:
best_global_trial = {stack: DataManager.load_best_trial_index(stack_m='atlasV3', stack_f=stack, 
                                          warp_setting=warp_setting, 
                                          classifier_setting_m=classifier_setting, 
                                          classifier_setting_f=classifier_setting, 
                                          downscale=32)
                     for stack in ['MD652']}

In [78]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d %%(trial_idx)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD652'],
                             'trial_idx': [best_global_trial['MD652']]},
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 40.330080 seconds


In [80]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d %%(trial_idx)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD652'],
                             'trial_idx': [best_global_trial['MD652']]},
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 95.636402 seconds


# Local align

In [81]:
classifier_setting = 37
warp_setting = 5

In [54]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

kwargs_list = [dict(stack='MD652', structures=shell_escape(json.dumps(all_known_structures_sided[f:l+1])))
for f,l in first_last_tuples_distribute_over(0, len(all_known_structures_sided)-1, 16)]

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d -s %%(structures)s --upstream_trial_idx %(global_trial_idx)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting,
                'global_trial_idx': best_global_trial},
                 kwargs_list=kwargs_list,        
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2047s for 5 trials.
# IC and SC are super slow...

LOCAL align ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 6625.116006 seconds


In [82]:
best_local_trials = {stack: DataManager.load_best_trial_index_all_structures(stack_m='atlasV3', stack_f='MD652', 
                                                      warp_setting=warp_setting, 
                                                      classifier_setting_m=classifier_setting, 
                                                      classifier_setting_f=classifier_setting, 
                                                      downscale=32)
for stack in ['MD652']}

rm -rf /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_sp5.txt && mkdir -p /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial
aws s3 cp s3://mousebrainatlas-data/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_sp5.txt /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_sp5.txt


Child returned 0
Child returned 1
0.34 seconds.
[Errno 2] No such file or directory: '/shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_sp5.txt'
Best trial file for structure sp5 is not found.
Child returned 0


rm -rf /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_outerContour.txt && mkdir -p /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial
aws s3 cp s3://mousebrainatlas-data/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_outerContour.txt /shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_outerContour.txt


Child returned 1
0.34 seconds.
[Errno 2] No such file or directory: '/shared/CSHL_registration_parameters/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial/atlasV3_down32_scoreVolume_clf_37_warp_5_MD652_down32_scoreVolume_clf_37_bestTrial_outerContour.txt'
Best trial file for structure outerContour is not found.


In [86]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d \'%%(trial_idx)s\' --upstream_trial_idx %%(global_trial_idx)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD652'],
                             'trial_idx': [json.dumps(best_local_trials['MD652'])],
                            'global_trial_idx': [best_global_trial['MD652']]
                            },
#                 kwargs_list=dict(stack=all_nissl_stacks),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 60 seconds

transform atlas ...Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 60.445804 seconds


In [95]:
warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m='atlasV3', stack_f='MD652',
                                    classifier_setting_m=classifier_setting,
                                    classifier_setting_f=classifier_setting,
                                    warp_setting=warp_setting,
                                    trial_idx=2, sided=True,
                                                                          structures=['7N_L'])

In [114]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d \'%%(trial_idx)s\'' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                  kwargs_list={'stack': ['MD652'],
                             'trial_idx': [json.dumps(best_local_trials['MD652'])]
                              },
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 105 seconds

visualize aligned atlas overlay ...Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 100.864477 seconds


# Transform locally transformed volumes back to atlas space

In [ ]:
classifier_setting = 2
warp_setting = 3

In [ ]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

# Measure Global Tx Confidence

In [ ]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

# Measure Local Tx Confidence

In [ ]:
warp_setting = 5
classifier_setting = 2

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins